# Publish geoscience objects from a RESQML file

This notebook shows how you can login to Evo and publish geoscience objects from a RESQML file to your chosen workspace.

In this first cell we create a ServiceManagerWidget which will open a browser window and ask you to login.

Once logged in, a widget will be displayed below allowing you to select an organisation and workspace to publish objects to.

__Required:__ You must fill in your `client_id` before running this cell.

In [ ]:
from evo.notebooks import ServiceManagerWidget

manager = await ServiceManagerWidget.with_auth_code(
    client_id="your-client-id", oidc_issuer="https://ims.bentley.com"
).login()

In the cell below we choose the RESQML file we want to publish and set its path in the `resqml_file` variable.

Then we choose an EPSG code to use for the Coordinate Reference System. The way the EPSG code is determined is as follows:
If there is a CRS associated with the Grid/Surface/etc to get the `epsg_code`, then use that. If not then use the model CRS.
And finally if there is no model CRS, use the default `epsg_code` passed in to the routine.

Then we call `convert_resqml`, passing it the RESQML file path, EPSG code, the service manager widget from above and finally a path we want the published objects to appear under.

Then we loop over the results from the `convert_resqml` function, printing out each object that was published to Evo.

_Note:_ The data for this sample is not included in the repository and the following two files will need to be down loaded to samples/data-converters/python/convert-resqml/data

    https://github.com/bp/resqpy/blob/master/example_data/block.epc
    https://github.com/bp/resqpy/blob/master/example_data/block.h5


In [ ]:
import os
import pprint

from evo.data_converters.resqml.importer import RESQMLConversionOptions, convert_resqml

resqml_file = os.path.join(os.getcwd(), "data/block.epc")
epsg_code = 32650
tags = {
    "TagName": "Tag value",
}

# Override the default option and export all cells, instead of just the active cells
# Override the memory threshold setting it to 1 GiB (the default is 8 GiB)
ONE_GB = 1 * 1024 * 1024 * 1024
options = RESQMLConversionOptions(active_cells_only=False, memory_threshold=ONE_GB)
objects_metadata = convert_resqml(
    filepath=resqml_file, epsg_code=epsg_code, service_manager_widget=manager, tags=tags, upload_path="notebook"
)

print()
print("These objects have now been published:")

for metadata in objects_metadata:
    pprint.pp(metadata, indent=4)

# Notes:
## Log messages
`Ignoring property obj_ContinuousProperty_2de3f074-524d-48a4-af75-832d71c708ef.xml 2de3f074-524d-48a4-af75-832d71c708ef, properties with multiple PatchOfValues are not supported`

**RESQML** supports multiple *PatchOfValue* elements for an attribute, however *resqpy* does not currently support this so the attribute has not been converted.

## options.memory_threshold

To calculate the grid geometry the *resqpy* `grid.corner_points` array is used, this array can be very large so the maximum size for this is specified in `RESQMLConversionOptions::memory_threshold`. Grids which would exceed this size will not be converted.


